In [ ]:
import keras
import os.path as osp
import tensorflow as tf
import sklearn.metrics
from tqdm import tqdm

In [ ]:
path = "/scratch/ssd004/scratch/jackk/birds_data"
model = keras.models.load_model(osp.join(path, 'EfficientNetB0-525-(224 X 224)- 98.97.h5'), custom_objects={'F1_score':'F1_score'}) 

In [ ]:
te_dset = tf.keras.utils.image_dataset_from_directory(osp.join(path, "test"), image_size=(224, 224))
va_dset = tf.keras.utils.image_dataset_from_directory(osp.join(path, "valid"), image_size=(224, 224))

In [ ]:
# rotate, crop and scale back up
def get_f1(dset, scale=20):
    full_labels = tf.zeros([0], dtype=tf.int32)
    full_preds = tf.zeros([0], dtype=tf.int64)
    for b in tqdm(dset):
        noise = tf.random.uniform(b[0].shape, minval=-scale, maxval=scale)
        preds = model(b[0] + noise)
        full_labels = tf.concat([full_labels, b[1]], 0)
        full_preds = tf.concat([full_preds, tf.math.argmax(preds, axis=-1)], 0)
    full_labels = full_labels.numpy()
    full_preds = full_preds.numpy()
    return sklearn.metrics.f1_score(full_preds, full_labels, average="micro")

In [ ]:
print(get_f1(te_dset, scale=50))
# print(get_f1(va_dset))